In [1]:
import pandas as pd
import numpy as np

In [2]:
CEO_DF = pd.read_csv('../outputs/CEO_DF.csv', index_col=False)

In [3]:
BOD_DF = pd.read_csv('../outputs/BOD_DF.csv', index_col=False)

In [4]:
CEO_DF['BECAMECEO'] = CEO_DF['BECAMECEO'].astype('datetime64[ns]')

In [5]:
x = CEO_DF['TDC1'].quantile([0.01]).values[0]
CEO_DF = CEO_DF[CEO_DF['TDC1'] >= x]

In [6]:
inner_merge_df = CEO_DF.merge(BOD_DF, how = 'inner',
                              on = ['TICKER', 'YEAR']).reset_index(drop=True)

In [7]:
inner_merge_df = inner_merge_df.drop(['Unnamed: 0_x','Unnamed: 0_y'], axis = 1)

# Determining ownership power

In [8]:
inner_merge_df['ownershipRatio'] = inner_merge_df['STOCK_AWARDS_x'] / inner_merge_df['total_STOCK_AWARDS']
median = inner_merge_df['ownershipRatio'].median()
inner_merge_df['ownershipPower'] = 0
inner_merge_df.ownershipPower[inner_merge_df.ownershipRatio > median] = 1
inner_merge_df['BECAMECEO'].dt.year

/var/folders/1t/r9hnt1ws3bg9x2szpm5f0ywm0000gn/T/ipykernel_11519/812312777.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inner_merge_df.ownershipPower[inner_merge_df.ownershipRatio > median] = 1


0       2003.0
1       2003.0
2       2011.0
3       2011.0
4       2013.0
         ...  
4693    2013.0
4694    2013.0
4695    2013.0
4696    2013.0
4697    2013.0
Name: BECAMECEO, Length: 4698, dtype: float64

# Creating variables for Expert powers

In [9]:
inner_merge_df['yearServed'] =  inner_merge_df['YEAR'] - inner_merge_df['BECAMECEO'].dt.year
inner_merge_df['prestiegePower'] = 0
inner_merge_df.prestiegePower[inner_merge_df.yearServed > median] = 1


/var/folders/1t/r9hnt1ws3bg9x2szpm5f0ywm0000gn/T/ipykernel_11519/3078248038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inner_merge_df.prestiegePower[inner_merge_df.yearServed > median] = 1


In [10]:
final_df = inner_merge_df[['GVKEY_x', 'TICKER','AGE', 'TDC1', 'YEAR','BECAMECEO',
                         'STOCK_AWARDS_x', 'STOCK_AWARDS_y', 'ownershipRatio',
                        'ownershipPower', 'yearServed', 'prestiegePower',
                          'size_category_x', 'MKVALT_x']]

# Exporting merged df

In [11]:
final_df.to_csv('../outputs/CEO_BOD_Merged.csv')